In [ ]:
import pandas as pd
import requests

from config import *

In [ ]:
years = ['2023', '2024', '2025']
params = {
    "latitude": COORDINATES["latitude"],
    "longitude": COORDINATES["longitude"],
    "hourly": "apparent_temperature",
    "timezone": "Europe/Moscow"
}
weather_dfs = {}

In [ ]:
for year in years:
    params['start_date'] = f'{year}-01-01'
    params['end_date'] = f'{year}-12-31'
    responses = requests.get(WEATHER_URL, params).json()

    time = responses['hourly']['time']
    date_list = pd.to_datetime(time)

    apparent_temperature = responses['hourly']['apparent_temperature']
    temp_list = pd.to_numeric(apparent_temperature)

    weather_df = pd.DataFrame.from_dict({
        'year': date_list.year, 
        'month': date_list.month, 
        'day': date_list.day, 
        'temp': temp_list})

    weather_df = weather_df.groupby(by=['year', 'month', 'day']).mean()
    weather_df = weather_df.mean()
    weather_dfs[year] = weather_df.reset_index()

In [ ]:
for key, val in weather_dfs:
    val.to_csv(f"{DATA_PATH['parsed']}weather_{key}.csv", sep=",", index=False)